In [1]:
'''

載入設定檔

'''

import json

secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# SQLITE3

In [3]:
import sqlite3

In [126]:
sql = '''
        select C_name0,(C_name1||' '||E_name1),(C_name2||' '||E_name2),(C_name3||' '||E_name3),url1,url2,url3
        from Movie_cf
        where C_name0 like '%{}%';
    '''
sql2 = '''
        select (C_name||' '||E_name),url
        from Movie_km
        where class2 like '{}'
        ORDER BY RANDOM()
        limit 3;
    '''
def get_3cf_movies(string):
    conn = sqlite3.connect('../data/movie_cf.db')
    cur = conn.cursor()
    if(cur.execute(sql.format(string))):
        out = cur.fetchone()
        if(out != None):
            output = []
            output.append(out[0])
            output.append(out[1]+"\n" + out[4])
            output.append(out[2]+"\n"+ out[5])
            output.append(out[3]+"\n" + out[6])
            conn.commit()
            conn.close()        
            return output
        else:
            conn.commit()
            conn.close()
            return 0
def get_3km_movies(string):
    conn = sqlite3.connect('../data/movie_km.db')
    cur = conn.cursor()

    if(cur.execute(sql2.format(string))):
        out = cur.fetchall()
        if(out != None):
            output = []
            output.append("為你推薦以下三部電影")
            output.append(out[0][0]+"\n" + out[0][1])
            output.append(out[1][0]+"\n" + out[1][1])
            output.append(out[2][0]+"\n" + out[2][1])
            conn.commit()
            conn.close()
            return output
        else:
            conn.commit()
            conn.close()
            return 0

In [6]:
import requests
from bs4 import BeautifulSoup

def movie():
    target_url = 'http://www.atmovies.com.tw/movie/next/0/'
    print('Start parsing movie ...')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    content = ""
    for index, data in enumerate(soup.select('ul.filmNextListAll a')):
        if index == 5:
            return content
        title = data.text.replace('\t', '').replace('\r', '')
        link = "http://www.atmovies.com.tw" + data['href']
        content += '{}\n{}\n'.format(title, link)
    return content


In [8]:
'''

Template Message
    分成四種
        Buttons
        Confirm
        Carousel
        Image Carousel

'''
from linebot.models import (
    MessageEvent, TemplateSendMessage 
)

from linebot.models.template import(
    ButtonsTemplate, 
    PostbackTemplateAction,
    MessageTemplateAction,
    URITemplateAction,
    DatetimePickerTemplateAction
)
from linebot.models.template import (
    CarouselTemplate, CarouselColumn
)


In [19]:
'''

Button篇


 '''


movie_km_buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url=secretFileContentJson.get("server_url")+'/images/eddie.jpg',
        title='恐怖驚悚',
        text='Please select',
        actions=[
            URITemplateAction(
                label='協同過濾',
                url='http://www.imdb.com/title/tt5308322'
            ),
            URITemplateAction(
                label='分類推薦',
                url='http://www.imdb.com/title/tt5308322'
            ),
            URITemplateAction(
                label='近期上映',
                url='http://www.imdb.com/title/tt5308322'
            )
        ]
    )
)



In [20]:
def get_3km_template(string):
    movie_km_buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url=secretFileContentJson.get("server_url")+'/images/eddie.jpg',
        title=string,
        text='Please select',
        actions=[
            URITemplateAction(
                label=get_3km_movies("恐怖驚悚")[0][0],
                url=get_3km_movies("恐怖驚悚")[0][1]
            ),
            URITemplateAction(
                label=get_3km_movies("恐怖驚悚")[1][0],
                url=get_3km_movies("恐怖驚悚")[1][1]
            ),
            URITemplateAction(
                label=get_3km_movies("恐怖驚悚")[2][0],
                url=get_3km_movies("恐怖驚悚")[2][1]
            )
        ]
    )
)
    return movie_km_buttons_template_message

In [10]:
from linebot.models.template import (
    CarouselTemplate, CarouselColumn
)

carousel_template_message = TemplateSendMessage(
    alt_text='Carousel template',
    template=CarouselTemplate(
        columns=[
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine.jpg' % server_url,
                title='恐怖',
                text='請選出你最感興趣的類別',
                actions=[
                    MessageTemplateAction(
                        label='恐怖驚悚',
                        text='恐怖驚悚'
                    ),
                    MessageTemplateAction(
                        label='殭屍',
                        text='殭屍'
                    ),
                    MessageTemplateAction(
                        label='現代風格',
                        text='現代風格'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine.jpg' % server_url,
                title='動作',
                text='請選出你最感興趣的類別',
                actions=[
                    MessageTemplateAction(
                        label='動作武打',
                        text='動作武打'
                    ),
                    MessageTemplateAction(
                        label='動作英雄',
                        text='動作英雄'
                    ),
                    MessageTemplateAction(
                        label='動作戰爭',
                        text='動作戰爭'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine.jpg' % server_url,
                title='喜劇',
                text='請選出你最感興趣的類別',
                actions=[
                    MessageTemplateAction(
                        label='家庭喜劇',
                        text='家庭喜劇'
                    ),
                    MessageTemplateAction(
                        label='愛情喜劇',
                        text='愛情喜劇'
                    ),
                    MessageTemplateAction(
                        label='動畫喜劇',
                        text='動畫喜劇'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine.jpg' % server_url,
                title='科幻',
                text='請選出你最感興趣的類別',
                actions=[
                    MessageTemplateAction(
                        label='外星科幻',
                        text='外星科幻'
                    ),
                    MessageTemplateAction(
                        label='科幻奇幻',
                        text='科幻奇幻'
                    ),
                    MessageTemplateAction(
                        label='科幻動作',
                        text='科幻動作'
                    )
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://%s/images/janine/janine2.jpg' % server_url,
                title='劇情',
                text='請選出你最感興趣的類別',
                actions=[
                    MessageTemplateAction(
                        label='劇情傳記',
                        text='劇情傳記'
                    ),
                    MessageTemplateAction(
                        label='懸疑推理',
                        text='懸疑推理'
                    ),
                    MessageTemplateAction(
                        label='社會寫實',
                        text='社會寫實'
                    )
                ]
            )
        ]
    )
)

In [2]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# create a instance for line
# line_bot_api 用來處理消息
# 用來接收外部消息
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body 驗證封包是否來自Line
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
    
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url='https://img.char.tw/uploads/2017/08/1503040202-f74a07c9f8983d9f821d92911c1d0def.jpg' ,
        title='電影推薦系統',
        text='Please select',
        actions=[
            MessageTemplateAction(
                label='協同過濾',
                text='協同過濾'
            ),
            MessageTemplateAction(
                label='分類推薦',
                text='分類推薦'
            ),
            MessageTemplateAction(
                label='近期上映',
                text='近期上映電影'
            )
        ]
    )
)
    
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.message.text == "電影":
        line_bot_api.reply_message(
            event.reply_token,
            buttons_template_message)
        return 0
    if event.message.text == "協同過濾":
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="輸入一部電影，找出最接近的三部"))
        return 0
    
    
    if event.message.text == "近期上映電影":
        content = movie()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0
    if event.message.text == "分類推薦":
        line_bot_api.reply_message(
            event.reply_token,
            carousel_template_message)
        return 0
    
    if (event.message.text in km_movie):
        content = get_3km_movies(event.message.text)
        km_buttons_template_message = 0
        km_buttons_template_message = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                  thumbnail_image_url='https://2646dfe4.ap.ngrok.io/images/ameica.jpg',
                  title=event.message.text,
                  text=content[0],
                  actions=[
                       URITemplateAction(
                           label=content[1].split("\n")[0],
                           uri=content[1].split("\n")[1]
                           ),
                       URITemplateAction(
                           label=content[2].split("\n")[0],
                           uri=content[2].split("\n")[1]
                           ),
                       URITemplateAction(
                           label=content[3].split("\n")[0],
                           uri=content[3].split("\n")[1]
                          )
                      ]
                   )
                )
        print(km_buttons_template_message)
        line_bot_api.reply_message(event.reply_token, km_buttons_template_message)
        return 0
    
    else: 
        content = get_3cf_movies(event.message.text)
        if(content != 0):
            print(content[0])
            cf_buttons_template_message = 0
            cf_buttons_template_message = TemplateSendMessage(
                alt_text='Buttons template',
                template=ButtonsTemplate(
                      thumbnail_image_url='https://2646dfe4.ap.ngrok.io/images/ameica.jpg',
                      title="你輸入的電影是",
                      text=content[0],
                      actions=[
                           URITemplateAction(
                               label=content[1].split("\n")[0],
                               uri=content[1].split("\n")[1]
                               ),
                           URITemplateAction(
                               label=content[2].split("\n")[0],
                               uri=content[2].split("\n")[1]
                               ),
                           URITemplateAction(
                               label=content[3].split("\n")[0],
                               uri=content[3].split("\n")[1]
                              )
                          ]
                       )
                    )
            print(cf_buttons_template_message)
            line_bot_api.reply_message(event.reply_token, cf_buttons_template_message)
            return 0
        if(content == 0):
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text="抱歉搜尋不到您要的電影，請再次搜尋"))
    

if __name__ == "__main__":
    app.run(host='0.0.0.0')